In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

In [2]:
url = 'https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo-zona-norte/'
c = requests.get(url).content

In [3]:
soup = BeautifulSoup(c)

## Criando uma lista com os preços dos alugueis

In [4]:
lista_precos_tags = soup.find_all('span', {'class':'price-tag-fraction'})

- For loop para obter a lista de preços

In [5]:
lista_precos = []
for tag in lista_precos_tags:
    lista_precos.append(tag.text)
lista_precos[0:5]

['900', '1.000', '1.299', '620', '799']

 - List comprehension

In [6]:
# for loop utilizando list compreehension
lista_precos  = [tag.text for tag in lista_precos_tags]
lista_precos[0:5]

['900', '1.000', '1.299', '620', '799']

In [17]:
len(lista_precos)

48

## Criando uma lista com a área e quantidade de quartos

In [7]:
lista_area_quartos_tags = soup.find_all('ul', {'class':'ui-search-card-attributes'})

In [8]:
lista_area_quartos = [info.text for info in lista_area_quartos_tags ]
lista_area_quartos[0:5]

['30 m² construídos1 quarto',
 '50 m² construídos1 quarto',
 '80 m² construídos2 quartos',
 '30 m² construídos1 quarto',
 '50 m² construídos1 quarto']

In [16]:
len(lista_area_quartos)

48

## Criando uma lista com os endereços

In [9]:
lista_enderecos_tags = soup.find_all('span' , {'class':'ui-search-item__group__element'})

In [10]:
lista_enderecos = [end.text for end in lista_enderecos_tags]
lista_enderecos = lista_enderecos[1::2]
lista_enderecos[:5]

['R Das Tâmaras, Vila Paulista, São Paulo Zona Norte, São Paulo',
 'Rua Campo Belo Do Sul 113, Parque Vitória, São Paulo Zona Norte, São Paulo',
 'Rua José Eduardo Raduan 5, Parque Edu Chaves, São Paulo Zona Norte, São Paulo',
 'Rua Rodolfo Bardela 335, Brasilândia, São Paulo Zona Norte, São Paulo',
 'Brandão Agrões 176, Jardim Peri, São Paulo Zona Norte, São Paulo']

In [15]:
len(lista_enderecos)

48

## Criando  função que retorna um DataFrame com as informaçoes obtidas anteriormente

In [21]:
lista_pag = []
for i in range(1,1537):
    num_1 ='_Desde_'
    num_2 = i
    num = '{}{}'.format(num_1,num_2)
    lista_pag.append(num)


lista_pag_1 = lista_pag[::48]
lista_pag_1[0] = ''

In [25]:
zonas = ['zona-norte', 'zona-sul', 'zona-leste', 'zona-oeste', 'centro']
URL='https://imoveis.mercadolivre.com.br/casas/aluguel/sao-paulo/sao-paulo-{}/{}'
paginas = lista_pag_1[0:7]

In [26]:
def scrap(url):
    c = requests.get(url).content
    soup = BeautifulSoup(c)
    lista_precos_tags = soup.find_all('span', {'class':'price-tag-fraction'})
    lista_precos  = [tag.text for tag in lista_precos_tags]
    lista_area_quartos_tags = soup.find_all('ul', {'class':'ui-search-card-attributes'})
    lista_area_quartos = [info.text for info in lista_area_quartos_tags ]
    lista_enderecos_tags = soup.find_all('span' , {'class':'ui-search-item__group__element'})
    lista_enderecos = [end.text for end in lista_enderecos_tags]
    lista_enderecos = lista_enderecos[1::2]
    df=convert_df(zona, lista_area_quartos, lista_enderecos, lista_precos)
    return df


def convert_df(zona, lista_area_quartos, lista_enderecos, lista_precos):
    zonas_lista = [zona]*len(lista_precos)
    data = {'zona': zonas_lista,
            'area_quartos': lista_area_quartos,
            'enderecos_lista':lista_enderecos,
            'aluguel': lista_precos}
    df = pd.DataFrame(data)
    return df

In [27]:
lista_dfs = []
for zona in zonas:
    for pag in paginas:
        url='https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-{}/{}'.format(zona,pag)
        print(url)
        df=scrap(url)
        lista_dfs.append(df)
        sleep(1)

https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_49
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_97
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_145
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_193
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_241
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-norte/_Desde_289
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-sul/
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_49
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_97
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_145
https://imoveis.mercadolivre.com.br/aluguel/sao-paulo/sao-paulo-zona-sul/_Desde_193
ht

In [28]:
df = pd.concat(lista_dfs)

In [29]:
df.head()

,zona,area_quartos,enderecos_lista,aluguel
0,zona-norte,60 m² construídos2 quartos,"Av Maestro Villa Lobos, Vila Gustavo, São Paul...",1.600
1,zona-norte,30 m² construídos1 quarto,"R Das Tâmaras, Vila Paulista, São Paulo Zona N...",900
2,zona-norte,51 m² construídos2 quartos,"Rua José Timotheo Da Silva, Jardim Peri, São P...",950
3,zona-norte,126 m² construídos3 quartos,"Rua Plinio Colás, Lauzane Paulista, São Paulo ...",2.500
4,zona-norte,55 m² construídos1 quarto,"Rua Doutor Gabriel Covelli, Parque Peruche, Sã...",900


In [30]:
df.to_csv('dados_aluguels_imoveis_ml_sao_paulo.csv', index =False , encoding = 'latin-1')

UnicodeEncodeError: 'latin-1' codec can't encode character '\u2019' in position 66: ordinal not in range(256)